# I feel like I'm in the flow

- https://adventofcode.com/2023/day/19

Part one seems to me to be a straightforward system of operator tests; just pass each part to the first workflow and have the workflow return the name of the next workflow or a sorting result.


In [1]:
import typing as t
from dataclasses import dataclass
from enum import Enum, IntEnum
from operator import gt, lt

OPERATORS = {">": gt, "<": lt}


class Category(IntEnum):
    x = 0
    m = 1
    a = 2
    s = 3


class Part(t.NamedTuple):
    x: int
    m: int
    a: int
    s: int

    @classmethod
    def from_line(cls, line: str) -> t.Self:
        kvpairs = (pair.split("=") for pair in line.strip("{}").split(","))
        return cls(**{k: int(v) for k, v in kvpairs})


class Result(Enum):
    accepted = "A"
    rejected = "R"

    @classmethod
    def from_target(cls, target: str) -> t.Self | str:
        try:
            return cls(target)
        except ValueError:
            return target


@dataclass
class Rule:
    op: t.Callable[[int, int], bool]
    category: Category
    value: int
    target: str | Result

    def __call__(self, part: Part) -> str | Result | None:
        return self.target if self.op(part[self.category], self.value) else None

    @classmethod
    def from_str(cls, rule: str) -> t.Self:
        expr, _, target = rule.partition(":")
        cat, op, value = expr.partition(">") if ">" in expr else expr.partition("<")
        return cls(OPERATORS[op], Category[cat], int(value), Result.from_target(target))


@dataclass
class Workflow:
    name: str
    rules: tuple[Rule, ...]
    else_: str | Result

    def __call__(self, part: Part) -> str | Result:
        return next(filter(None, (rule(part) for rule in self.rules)), self.else_)

    @classmethod
    def from_line(cls, line: str) -> t.Self:
        name, _, rules_and_fallback = line.rstrip("}").partition("{")
        *rules, fallback = rules_and_fallback.split(",")
        fallback = Result.from_target(fallback)
        return cls(name, tuple(map(Rule.from_str, rules)), fallback)


@dataclass
class System:
    workflows: dict[str, Workflow]

    def __call__(self, part: Part) -> bool:
        workflow = self.workflows["in"]
        while True:
            match workflow(part):
                case Result.accepted:
                    return True
                case Result.rejected:
                    return False
                case str(target):
                    workflow = self.workflows[target]

    @classmethod
    def from_text(cls, text: str) -> t.Self:
        return cls(
            {(wf := Workflow.from_line(line)).name: wf for line in text.splitlines()}
        )


def parse(text: str) -> tuple[System, list[Part]]:
    workflows, parts = text.split("\n\n")
    return System.from_text(workflows), [
        Part.from_line(line) for line in parts.splitlines()
    ]


test_workflows_and_parts = """\
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""

test_system, test_parts = parse(test_workflows_and_parts)
sum(map(sum, filter(test_system, test_parts)))

19114

In [2]:
import aocd

system, parts = parse(aocd.get_data(day=19, year=2023))
print("Part 1:", sum(map(sum, filter(system, parts))))

Part 1: 409898


# Homing in on the goldilocks range

Part two is a bit more interesting; instead of a single part we are now dealing with a range of values for each part category. Workflow rules sort these ranges into those that match and don't match.

I've created replacements for each of my classes to handle ranges now:

- Parts are replaced by a `PartsRange` class, which hold `range()` objects for each category
- Rules have become `RangeRule` instances, and they return their target with two `PartsRange` objects, one for where the rule applies, and one where it doesn't.
- The `RangeWorkflow` class models a workflow, and yields tuples with the next workflow name or a result, together with the `PartsRange` that this applies to. It'll apply the next rule to the _other_ `PartsRange`, where the current rule didn't apply, until the end of the rule list is reached and we return the alternative result with the remainder.
- Finally, the `RangeSystem` keeps a queue of `RangeWorkflow` and `PartsRange` objects, and runs them until the queue is empty. Each workflow produces an iterable of targets and `PartsRange` objects, and if the target is a workflow name, then that workflow is added to the queue together with the constrained `PartsRange`. For _Accepted_ result, we can update a running total (the product of the range lengths if accepted), and we can just ignore _Rejected_ ranges.

This completes part two very nice and fast, in about 2.5 ms.


In [3]:
from collections import deque
from math import prod


class PartsRange(t.NamedTuple):
    x: range = range(1, 4001)
    m: range = range(1, 4001)
    a: range = range(1, 4001)
    s: range = range(1, 4001)

    @property
    def size(self) -> int:
        return prod(map(len, self))


@dataclass
class RangeRule:
    op: t.Literal[">", "<"]
    category: Category
    value: int
    target: str | Result

    def __call__(self, part: PartsRange) -> tuple[str | Result, PartsRange, PartsRange]:
        cat, bound = self.category, self.value
        r = part[cat]
        f, t = r.start, r.stop
        if self.op == "<":
            tr = range(0) if f >= bound else range(f, min(t, bound))
            fr = range(0) if t <= bound else range(max(f, bound), t)
        else:  # ">"
            tr = range(0) if t <= bound + 1 else range(max(f, bound + 1), t)
            fr = range(0) if f > bound else range(f, min(t, bound + 1))
        return (
            self.target,
            part._replace(**{cat.name: tr}),
            part._replace(**{cat.name: fr}),
        )

    @classmethod
    def from_rule(cls, rule: Rule) -> t.Self:
        op = ">" if rule.op is gt else "<"
        return cls(op, rule.category, rule.value, rule.target)


@dataclass
class RangeWorkflow:
    name: str
    rules: tuple[RangeRule, ...]
    else_: str | Result

    def __call__(
        self, parts_range: PartsRange
    ) -> t.Iterator[tuple[str | Result, PartsRange]]:
        for rule in self.rules:
            res, true_range, parts_range = rule(parts_range)
            yield res, true_range
        yield self.else_, parts_range

    @classmethod
    def from_workflow(cls, wf: Workflow) -> t.Self:
        return cls(wf.name, tuple(map(RangeRule.from_rule, wf.rules)), wf.else_)


@dataclass
class RangeSystem:
    workflows: dict[str, RangeWorkflow]

    def __call__(self) -> int:
        wfs = self.workflows
        todo: deque[tuple[RangeWorkflow, PartsRange]] = deque(
            [(wfs["in"], PartsRange())]
        )
        total = 0
        while todo:
            workflow, parts_range = todo.popleft()
            for res, pr in workflow(parts_range):
                match res:
                    case Result.accepted:
                        total += pr.size
                    case Result.rejected:
                        pass
                    case str(target):
                        todo.append((wfs[target], pr))
        return total

    @classmethod
    def from_system(cls, system: System) -> t.Self:
        wfs = {
            name: RangeWorkflow.from_workflow(wf)
            for name, wf in system.workflows.items()
        }
        return cls(wfs)


test_range_system = RangeSystem.from_system(test_system)
assert test_range_system() == 167409079868000

In [4]:
range_system = RangeSystem.from_system(system)
print("Part 2:", range_system())

Part 2: 113057405770956
